![banner](../logo/banner.png)

## 3.7 Salidas analogicas (PWM)

Los pines PWM son utilizados como un reemplazo a un DAC, es decir, se toma como una salida analogica. Los pines que se pueden utilizar como salida analogica (PWM) son las que tienen el simbolo `~`, en la placa.


![board_pwm](imgs/board_pwm.png)

Los cuales son los pines:

Pines|
:-:|
3|
5|
6|
9|
10|
11|

Esto significa que podemos tener 6 salidas analogicas para poder controlar cualquier elemento que soporte este control.

---

## 3.7.1 Configuracion de salidas PWM

Vamos a realizar la configuracion basica del PWM

```python
from pyfirmata import Arduino, util
from time import sleep
 
PORT = 'COM#'
board = Arduino(PORT)

# Configuramos el pin que sera utilizado como PWM
pin_analog = board.get_pin('d:6:p')

# Mando el valor que necesito al pin 6 con un ciclo de trabajo del 50%
pin_analog.write(0.5)

```

**Ejemplo 1: Vamos a controlar la intensidad de un LED con una salida PWM**

![pwm basico](imgs/pwm_1.png)

```python
from pyfirmata import Arduino
from time import sleep

PORT = 'COM#'
board = Arduino(PORT)

# Configuramos el pin que sera utilizado como PWM
pwm = board.get_pin('d:3:p')

while True:

    for i in range(0, 101,3):
        value = i/100
        print(value)
        pwm.write(value)
        sleep(0.2)
```

---
**Ejemplo 2: Vamos a controlar el intensidad de un LED con 2 botones, uno que incremente y con otro que decremente la intesidad**

![pwm btn](imgs/pwm_btn.png)

```python
from pyfirmata import Arduino, util
from time import sleep

PORT = 'COM#'
board = Arduino(PORT)

util.Iterator(board).start()

# pines en donde estan los botones
DOWN = 4
UP = 5

up = board.get_pin(f'd:{str(UP)}:i')
down = board.get_pin(f'd:{str(DOWN)}:i')

# Configuramos el pin que sera utilizado como PWM
led = board.get_pin('d:3:p')

# variable me indica el valor del pwm
intensidad = 0

while True:

    if up.read():
        if not (intensidad >= 99):
            intensidad += 3
            led.write(intensidad/100)
            print(intensidad/100)
            sleep(0.25)

    if down.read():
        if not (intensidad <= 0):
            intensidad -= 3
            led.write(intensidad/100)
            print(intensidad/100)
            sleep(0.25)
```

![pwm led btns](imgs/pwm_led_btn.png)

---
**Ejemplo 3: Vamos a controlar el intensidad de un LED con potenciometro la variar su intensidad.**

![](imgs/pwm_adc_1.png)


```python
from pyfirmata import Arduino, util
from time import sleep

PORT = 'COM#'
board = Arduino(PORT)

util.Iterator(board).start()

# pines en donde estan los botones
POT = 0
# configuro la entrada del potenciometro
pot = board.get_pin(f'a:{str(POT)}:i')

# Configuramos el pin que sera utilizado como PWM
LED = 3
led = board.get_pin(f'd:{str(LED)}:p')

sleep(1) #se estabiliza la configuracion

while True:

    intensidad = pot.read()
    sleep(0.25)

    if intensidad == None:
        continue

    led.write(intensidad)
    print(intensidad)
```

![pot led](imgs/pwm_pot_led.png)

## 3.7.2 Servomotor

El servomotor se mueve con base a una señal PWM, pero la libreria nos ahorra el trabajo para generar el desplazamiento.

El servomotor que utilizaremos el SG-90, es un servomotor basico.

![servo sg90](imgs/sg90.jpg)

Este motor necesita la siguiente señal para poder generar su desplazamiento:

![signal servo](imgs/pwm_servo_data.png)


### 3.7.2.1 Configurando Servo

Los pines que se pueden usar para el control de servo son los correspondientes al `PWM`.

**Un servomotor se puede dezplar desde 0° hasta 180°. Por default siempre esta en 90°. Pero por default se considera 0°, dado que gira hasta 90° y -90°.**

Para el control del Servomotor la libreria de pyfirmata contiene su configuracion. 

```python
from pyfirmata import Arduino, util
from time import sleep

PORT = 'COM#'
board = Arduino(PORT)

# Configuramos el pin que sera utilizado como PWM
servo = board.get_pin('d:3:s') # el tercer parametro define que tipo sera la salida
sleep(1)

while True:
    angulo = 0
    servo.write(angulo) # el servo se movera al angulo indicado
```

---

**Ejemplo 1: Controlar un Servomotor, se debe colocar en diversos angulos y repetir de manera indfinida. Los angulos son 0°,30°,60°,90°,120°,160°,180°**

```python
from pyfirmata import Arduino, util
from time import sleep

PORT = 'COM#'
board = Arduino(PORT)

# Configuramos el pin que sera utilizado como PWM
servo = board.get_pin('d:3:s') # el tercer parametro define que tipo sera la salida
sleep(1)

while True:
    angulo = 0
    servo.write(angulo)
    sleep(2)
    angulo = 30
    servo.write(angulo)
    sleep(2)
    angulo = 60
    servo.write(angulo)
    sleep(2)
    angulo = 90
    servo.write(angulo)
    sleep(2)
    angulo = 120
    servo.write(angulo)
    sleep(2)
    angulo = 160
    servo.write(angulo)
    sleep(2)
    angulo = 180
    servo.write(angulo)
    sleep(2)
```

![simul_servo](imgs/servo1.png)

---

**Ejemplo 2: Hacer girar un servomotor en funcion de la posicion de un potenciometro. Es decir, si esta en 0 el potenciomentro, sera 0°; en 0.5 sera 90° y al 100% seran 180°.**

![servo pot](imgs/servo_pot.png)

```python
from pyfirmata import Arduino, util
from time import sleep

PORT = 'COM#'
board = Arduino(PORT)

util.Iterator(board).start()

# Config pot
POT = 0
# configuro la entrada del potenciometro
pot = board.get_pin(f'a:{str(POT)}:i')

# config Servo
SERVO = 3
servo = board.get_pin(f'd:{str(SERVO)}:s')

sleep(1) # time to wait

while True:

    value = pot.read()
    sleep(0.25)

    if value == None:
        continue

    ang = value * 180
    servo.write(ang)
```

---
**Ejemplo 3: Cambiar el angulo de un servomotor con botones, es decir, con uno incrementa su posicion y con el segundo la decrementa**

![servo btn](imgs/servo_btn.png)

```python
from pyfirmata import Arduino, util
from time import sleep

PORT = '/tmp/ttyS1'
board = Arduino(PORT)

util.Iterator(board).start()

# Configuro botones
DOWN = 8
UP = 9

down = board.get_pin(f'd:{str(DOWN)}:i')
up = board.get_pin(f'd:{str(UP)}:i')
# configuro la entrada del potenciometro
POT = 3
pot = board.get_pin(f'a:{str(POT)}:i')

# config Servo
SERVO = 3
servo = board.get_pin(f'd:{str(SERVO)}:s')

sleep(1) # time to wait

step = 18
angulo = 0

while True:

    if down.read():
        if angulo <= 0:
            continue
        angulo -= step
        print(f'Incremento angulo {angulo}')
        servo.write(angulo)
        sleep(0.25)

    if up.read():
        if angulo >= 180:
            continue

        angulo += step
        print(f'Decremento el angulo {angulo}')
        servo.write(angulo)
        sleep(0.25)
```

---

## Tarea - Salidas Analogicas

1. Control de velocidad de un motor DC, se controlara la velocidad del motor atraves de un potenciometro. Si el potenciometro esta en 0 el motor debe estar apagado, si esta al 50% el motor girara a la mitad de su velocidad, cuando llegue a 100% el motor debe girar a su maxima velocidad.
2. Control de la intesidad de varias luces, se tendran 3 leds cada uno controlado con su respectivo potenciometro para hacer variar la intensidad de la luz de cada uno. Todos de manera independiente.
3. Control de la intesidad de varias luces, se tendran 3 luces. 2 leds cada uno controlado con su respectivo potenciometro para hacer variar la intensidad de la luz de cada uno, un tercero controlado con botones, uno para incrementar y otro para decrementar la intensidad. Todos de manera independiente.
4. Utilizando un servomotor que se utilizara para controlar la pluma de la entrada de estacionamiento. Se simula un sensor infrarrojo para detectar el auto y se levante la pluma (180°), cuando no detecte el auto, bajara la pluma (0°)

---

Realizado por Docente: [Alejandro Leyva](https://www.alejandro-leyva.com/)

[Mecatrónica 85](https://mecatronica85.com/)

[fb/mecatronica85](https://www.facebook.com/mecatronica85)